<a href="https://colab.research.google.com/github/SabrineRiahi/Key_Localisation_FloydHub/blob/master/Darknet21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Darknet

In [19]:
!git clone https://github.com/pjreddie/darknet
%cd ./darknet
!make

fatal: destination path 'darknet' already exists and is not an empty directory.
/content/darknet
make: Nothing to be done for 'all'.


# Training yolo on VOC


## Downloading datas

In [0]:
!wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
!wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!wget https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
!tar xf VOCtrainval_11-May-2012.tar
!tar xf VOCtrainval_06-Nov-2007.tar
!tar xf VOCtest_06-Nov-2007.tar

In [11]:
# Generate labels
%cd /content/darknet/data
!wget https://pjreddie.com/media/files/voc_label.py
!python voc_label.py

/content/darknet/data
--2019-11-09 12:24:17--  https://pjreddie.com/media/files/voc_label.py
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2042 (2.0K) [application/octet-stream]
Saving to: ‘voc_label.py’

voc_label.py        100%[===================>]   1.99K  --.-KB/s    in 0s      

2019-11-09 12:24:18 (459 MB/s) - ‘voc_label.py’ saved [2042/2042]



## Adding key's data 

In [0]:
# Adding key's images
%cd /content
!git clone https://github.com/belarbi2733/keras_yolov3

%cd /content/keras_yolov3
!wget https://github.com/belarbi2733/keras_yolov3/releases/download/1/key_wb.zip
!wget https://github.com/belarbi2733/keras_yolov3/releases/download/1/bckgrnd.zip
!unzip bckgrnd.zip
!unzip key_wb.zip
!python keys_with_background.py --keys "key_wb" --background "bckgrnd" --output "/content/darknet/data/VOCdevkit/VOC2007/img"

In [0]:
# Rename annotations.csv file into txt file
%cd /content/darknet/data/VOCdevkit/VOC2007/img
import os
pre, ext = os.path.splitext("/content/darknet/data/VOCdevkit/VOC2007/img/annotations.csv")
os.rename("annotations.csv", pre + ".txt")

In [74]:
# Generate 1 file "annotation.txt" for each image in the right directory
%cd /content/darknet/data/VOCdevkit/VOC2007/img
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/annotation_pythonv3.py
!python annotation_pythonv3.py

/content/darknet/data/VOCdevkit/VOC2007/img
--2019-11-09 15:59:10--  https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/annotation_pythonv3.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1223 (1.2K) [text/plain]
Saving to: ‘annotation_pythonv3.py.1’

annotation_pythonv3 100%[===================>]   1.19K  --.-KB/s    in 0s      

2019-11-09 15:59:10 (77.0 MB/s) - ‘annotation_pythonv3.py.1’ saved [1223/1223]



In [58]:
# Generate annotation_train.txt and annotation_test.txt == Directories of file img of keys
%cd /content/darknet/data/VOCdevkit/VOC2007/img
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/annotation_train_test.py
!python annotation_train_test.py

/content/darknet/data/VOCdevkit/VOC2007/labels
--2019-11-09 15:44:00--  https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/annotation_train_test.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 723 [text/plain]
Saving to: ‘annotation_train_test.py’

annotation_train_te 100%[===================>]     723  --.-KB/s    in 0s      

2019-11-09 15:44:00 (161 MB/s) - ‘annotation_train_test.py’ saved [723/723]



In [61]:
# Concatenate annotation_train.txt + 2007_train.txt = train.txt
filenames = ['/content/darknet/data/VOCdevkit/VOC2007/img/annotation_train.txt', '/content/darknet/data/2007_train.txt']
with open('/content/darknet/data/train.txt', 'w') as outfile : # Path to output file
    for fname in filenames :
        with open(fname) as infile :
            for line in infile :
                outfile.write(line)
# Concatenate annotation_test.txt + 2007_test.txt = test.txt
filenames = ['/content/darknet/data/VOCdevkit/VOC2007/img/annotation_test.txt', '/content/darknet/data/2007_test.txt']
with open('/content/darknet/data/test.txt', 'w') as outfile : # Path to output file
    for fname in filenames :
        with open(fname) as infile :
            for line in infile :
                outfile.write(line)
!mv /content/darknet/data/VOCdevkit/VOC2007/img/train.txt /content/darknet/data
!mv /content/darknet/data/VOCdevkit/VOC2007/img/test.txt /content/darknet/data

mv: cannot stat 'train.txt': No such file or directory
mv: cannot stat 'test.txt': No such file or directory


## REM : Modify files .voc .names .cfg

In [0]:
# modifier .cfg : nb classes, nb filtes
# modifier .names : 81 = clés
# modifier .voc : paths
# modifier .txt : path of img data test/train

## Downloading pretrained convolutionnal  weights

In [65]:
%cd /content/darknet
!wget https://pjreddie.com/media/files/darknet53.conv.74

/content/darknet
--2019-11-09 15:51:05--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  20.0MB/s    in 8.4s    

2019-11-09 15:51:14 (18.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



## Training model

In [67]:
# voc.names -> data
%cd /content/darknet/data
!rm voc.names
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/voc.names
# voc.data, yolov3.cfg -> cfg
%cd /content/darknet/cfg
!rm voc.data
!rm yolov3.cfg
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/voc.data
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/yolov3.cfg

/content/darknet/data
--2019-11-09 15:52:08--  https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/voc.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138 [text/plain]
Saving to: ‘voc.names’

voc.names           100%[===================>]     138  --.-KB/s    in 0s      

2019-11-09 15:52:08 (34.8 MB/s) - ‘voc.names’ saved [138/138]

/content/darknet/cfg
--2019-11-09 15:52:14--  https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/voc.data
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
L

In [75]:
%cd /content/darknet
!./darknet detector train cfg/voc.data cfg/yolov3.cfg darknet53.conv.74

/content/darknet
yolov3
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    

## Testing model

In [0]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

%cd /darknet
#!wget https://pjreddie.com/media/files/yolov3.weights
!./darknet detect cfg/yolov3.cfg backup/yolov3_last.weights data/....jpg
imShow('predictions.jpg')